### Das ausfetzen eines venv mit der transformers-Bibliothek war auf unserem Windows Betriebssystem leider nicht möglich, weshalb ein conda enviroment erstellt werden musste. Alle ausgeführten Befehle wurde in der transformer_requirements.txt hinterlegt. Die Automatische erstellung der requirements datei beinhaltete aus unerfindlichen Gründen relative Pfade wie "ipywidgets @ file:///home/conda/feedstock_root/build_artifacts/ipywidgets_1671720089366/work".

In [1]:
import transformers
from transformers import AutoTokenizer
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
import evaluate
import numpy as np
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import pipeline
import pandas as pd
import thinc_gpu_ops
transformers.logging.set_verbosity_warning()

c:\Users\monox\anaconda3\envs\transformer_env\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# load Train.csv and Test.csv into same datasetdic

data_files = {"train": "train.csv", "test": "test.csv"}
dataset = load_dataset("./data/IMDBsentiment/", data_files=data_files)

Using custom data configuration IMDBsentiment-f571e8f9095c6190


Extracting data files: 100%|██████████| 2/2 [00:00<00:00, 400.16it/s]
Generating train split: 0 examples [00:00, ? examples/s]D:\THM\ANLP\lib\site-packages\datasets\download\streaming_download_manager.py:727: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.read_csv(xopen(filepath_or_buffer, "rb", use_auth_token=use_auth_token), **kwargs)
Generating test split: 0 examples [00:00, ? examples/s]            D:\THM\ANLP\lib\site-packages\datasets\download\streaming_download_manager.py:727: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.read_csv(xopen(filepath_or_buffer, "rb", use_auth_token=use_auth_token), **kwargs)


Dataset csv downloaded and prepared to C:/Users/jonas/.cache/huggingface/datasets/csv/IMDBsentiment-f571e8f9095c6190/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


100%|██████████| 2/2 [00:00<00:00, 166.69it/s]


In [3]:
# reduce dataset train to 100 samples
dataset['train'] = dataset['train'].select(range(100))
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 5
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 5000
    })
})

In [4]:
# load bert-base-uncased tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [5]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [6]:
tokenized_imdb = dataset.map(preprocess_function, batched=True)

100%|██████████| 5/5 [00:02<00:00,  1.98ba/s]


In [7]:
tokenized_imdb["train"][:1]

{'text': ['I grew up (b. 1965) watching and loving the Thunderbirds. All my mates at school watched. We played "Thunderbirds" before school, during lunch and after school. We all wanted to be Virgil or Scott. No one wanted to be Alan. Counting down from 5 became an art form. I took my children to see the movie hoping they would get a glimpse of what I loved as a child. How bitterly disappointing. The only high point was the snappy theme tune. Not that it could compare with the original score of the Thunderbirds. Thankfully early Saturday mornings one television channel still plays reruns of the series Gerry Anderson and his wife created. Jonatha Frakes should hand in his directors chair, his version was completely hopeless. A waste of film. Utter rubbish. A CGI remake may be acceptable but replacing marionettes with Homo sapiens subsp. sapiens was a huge error of judgment.'],
 'label': [0],
 'input_ids': [[101,
   1045,
   3473,
   2039,
   1006,
   1038,
   1012,
   3551,
   1007,
   

In [8]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [9]:
accuracy = evaluate.load("accuracy")

In [10]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [11]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [12]:
model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [13]:
training_args = TrainingArguments(
    output_dir="trained_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_imdb["train"],
    eval_dataset=tokenized_imdb["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
D:\THM\ANLP\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 5
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2


Epoch,Training Loss,Validation Loss


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 5000
  Batch size = 16
Saving model checkpoint to trained_model\checkpoint-1
Configuration saved in trained_model\checkpoint-1\config.json
Model weights saved in trained_model\checkpoint-1\pytorch_model.bin
tokenizer config file saved in trained_model\checkpoint-1\tokenizer_config.json
Special tokens file saved in trained_model\checkpoint-1\special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 5000
  Batch size = 1

TrainOutput(global_step=2, training_loss=0.9597360491752625, metrics={'train_runtime': 224.6258, 'train_samples_per_second': 0.045, 'train_steps_per_second': 0.009, 'total_flos': 2261110632000.0, 'train_loss': 0.9597360491752625, 'epoch': 2.0})

In [21]:
text = "This movie is nice"

classifier = pipeline("sentiment-analysis", model="./trained_model/checkpoint-2/")
print(classifier(text))

loading configuration file ./trained_model/checkpoint-2/config.json
Model config BertConfig {
  "_name_or_path": "./trained_model/checkpoint-2/",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "NEGATIVE",
    "1": "POSITIVE"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "NEGATIVE": 0,
    "POSITIVE": 1
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.21.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading configuration file ./trained_model/check

[{'label': 'POSITIVE', 'score': 0.7436841130256653}]
